<a href="https://colab.research.google.com/github/oztuncbilek/urban-heat-island/blob/main/notebooks/01_data_acquisition_geemap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Acquisition

In [1]:
!pip install geemap google-cloud-storage rasterio matplotlib > /dev/null 2>&1

In [2]:
import ee
import geemap
import os
from google.colab import drive
import rasterio
import matplotlib.pyplot as plt


drive.mount('/content/drive')
work_dir = '/content/drive/MyDrive/UHI-Detection-Analysis/data/raw/'
os.makedirs(work_dir, exist_ok=True)


Mounted at /content/drive


In [4]:
# GEE kimlik doğrulama
ee.Authenticate()
ee.Initialize(project='manifest-pride-258211')

**MODIS for selecting hotest days through 10 year**

In [35]:
def get_modis_hottest_days(start_year=2014, end_year=2024):

    """
    Finds the hottest cloud-free summer days (May 15 – Sep 15) for Hamburg using MODIS LST data.
    Returns a DataFrame with date, LST in Kelvin, and Celsius.
    """

    # Create a 5 km buffer around Hamburg coordinates
    hamburg = ee.Geometry.Point(9.9937, 53.5511).buffer(5000)
    results = {}

    for year in range(start_year, end_year + 1):
        print(f"\nProcessing year {year}...")

        try:
            # Load MODIS LST Day 1km collection
            modis = ee.ImageCollection('MODIS/061/MOD11A1') \
                .filterBounds(hamburg) \
                .filterDate(f'{year}-05-15', f'{year}-09-15')
                # .filter(ee.Filter.lt('QC_Day', 2))  # Optional: strict quality filter

            # Function to extract mean LST and acquisition date
            def compute_lst(img):
                mean_lst = img.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=hamburg,
                    scale=1000
                ).get('LST_Day_1km')
                return ee.Feature(None, {
                    'lst': mean_lst,
                    'date': img.date().format('YYYY-MM-dd')
                })

            # Map function over collection, and filter out null results
            lst_features = modis.map(compute_lst).filter(
                ee.Filter.notNull(['lst'])
            )

            # Check if any valid images remain
            if lst_features.size().getInfo() == 0:
                print("No valid LST data available, skipping.")
                continue

            # Sort by LST descending and pick the hottest
            hottest = ee.Feature(lst_features.sort('lst', False).first())

            date = hottest.get('date').getInfo()
            lst_kelvin = hottest.get('lst').getInfo()

            # Some years might return null
            if lst_kelvin is None:
                print("No temperature value found, skipping.")
                continue

            lst_celsius = lst_kelvin * 0.02 - 273.15

            results[year] = {
                'date': date,
                'lst_kelvin': lst_kelvin,
                'lst_celsius': round(lst_celsius, 2)
            }

            print(f"Hottest day: {date} | LST: {lst_celsius:.2f} °C")

        except Exception as e:
            print(f"Error: {str(e)}")
            continue

    return pd.DataFrame.from_dict(results, orient='index')

print("\Extracting MODIS data...")
df = get_modis_hottest_days()
print("\SResults:")
print(df)

\Extracting MODIS data...

Processing year 2014...
Hottest day: 2014-07-04 | LST: 34.83 °C

Processing year 2015...
Hottest day: 2015-07-05 | LST: 38.17 °C

Processing year 2016...
Hottest day: 2016-06-05 | LST: 35.11 °C

Processing year 2017...
Hottest day: 2017-05-27 | LST: 32.62 °C

Processing year 2018...
Hottest day: 2018-07-27 | LST: 37.34 °C

Processing year 2019...
Hottest day: 2019-06-30 | LST: 39.26 °C

Processing year 2020...
Hottest day: 2020-06-27 | LST: 36.38 °C

Processing year 2021...
Hottest day: 2021-06-17 | LST: 37.72 °C

Processing year 2022...
Hottest day: 2022-07-20 | LST: 39.18 °C

Processing year 2023...
Hottest day: 2023-07-15 | LST: 34.22 °C

Processing year 2024...
Hottest day: 2024-06-25 | LST: 33.50 °C
\SResults:
            date    lst_kelvin  lst_celsius
2014  2014-07-04  15399.234460        34.83
2015  2015-07-05  15566.021583        38.17
2016  2016-06-05  15413.236373        35.11
2017  2017-05-27  15288.740653        32.62
2018  2018-07-27  15524.5183

In [ ]:
# Hamburg koordinatları
hamburg = ee.Geometry.Point(9.99, 53.55)

# Landsat 8 koleksiyonunu filtrele (2015 ve 2024 yaz ayları)
def get_landsat_data(year):
    collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterBounds(hamburg) \
        .filterDate(f'{year}-06-01', f'{year}-08-31') \
        .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
        .sort('CLOUD_COVER') \
        .mean()
    return collection

# LST hesaplama fonksiyonu (basitleştirilmiş)
def calculate_lst(image):
    lst = image.expression(
        '(TIRS1 * 0.00341802 + 149.0) - 273.15',  # Kelvin'den Celsius'a
        {'TIRS1': image.select('ST_B10')}
    ).rename('LST')
    return lst

# 2015 ve 2025 için LST görüntülerini al
lst_2015 = calculate_lst(get_landsat_data(2015))
lst_2024 = calculate_lst(get_landsat_data(2024))

In [ ]:
# Interactive harita oluştur
Map = geemap.Map(center=[53.55, 9.99], zoom=12)

# LST görüntülerini haritaya ekle (renk paleti: 'inferno' veya 'coolwarm')
vis_params = {
    'min': 20,  # Min LST (°C)
    'max': 40,  # Max LST (°C)
    'palette': ['blue', 'green', 'yellow', 'red']  # Veya 'inferno'
}

Map.addLayer(lst_2015, vis_params, 'LST 2015')
Map.addLayer(lst_2024, vis_params, 'LST 2025')

# Layer kontrol paneli ekle
Map.addLayerControl()
Map

Map(center=[53.55, 9.99], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [ ]:
import geemap
import ipywidgets as widgets
from IPython.display import display

# Harita oluştur
Map = geemap.Map(center=[53.55, 9.99], zoom=12)

# Görselleştirme parametreleri
vis_params = {
    'min': 20,
    'max': 40,
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Yıllar listesi
years = [2015, 2024]

# Play düğmesi
play = widgets.Play(
    value=2015,
    min=2015,
    max=2024,
    step=9,
    interval=1500,  # milisaniye cinsinden (1.5 saniye)
    description="Oynat",
    disabled=False
)

# Slider
slider = widgets.IntSlider(value=2015, min=2015, max=2024, step=9, description='Yıl:')

# Play düğmesi ve slider'ı birbirine bağla
widgets.jslink((play, 'value'), (slider, 'value'))

# Slider değişince haritayı güncelle
def update_map(change):
    Map.layers = Map.layers[:1]  # Önceki katmanları temizle (sadece base map kalsın)
    if slider.value == 2015:
        Map.addLayer(lst_2015, vis_params, 'LST 2015')
    elif slider.value == 2024:
        Map.addLayer(lst_2024, vis_params, 'LST 2024')
    Map.addLayerControl()

slider.observe(update_map, names='value')

# Başlangıç katmanı ekle
Map.addLayer(lst_2015, vis_params, 'LST 2015')
Map.addLayerControl()

# Arayüzü göster
display(widgets.HBox([play, slider]))
Map



Map(center=[53.55, 9.99], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

LayerException: layer not on map: EELeafletTileLayer(attribution='Google Earth Engine', max_zoom=24, name='LST 2015', options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://earthengine.googleapis.com/v1/projects/manifest-pride-258211/maps/e824458757b13379a948d5a4f00960db-6038a256ab78da6c0a0874bb889094b3/tiles/{z}/{x}/{y}')

In [ ]:
# GeoTIFF olarak export etme fonksiyonu (GÜNCEL)
def export_to_drive(image, name, folder):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=name,
        folder=folder.replace('/content/drive/MyDrive/', ''),  # GEE için göreli yol
        fileNamePrefix=name,
        scale=30,
        region=hamburg.buffer(5000).bounds(),
        fileFormat='GeoTIFF'
    )
    task.start()
    return task

work_dir = 'raw'

# Drive'a kaydet (work_dir kullanarak)
export_to_drive(lst_2015, 'LST_2015_Hamburg', work_dir)
export_to_drive(lst_2024, 'LST_2024_Hamburg', work_dir)

# Task'lerin tamamlanmasını bekle
import time
while True:
    tasks = ee.batch.Task.list()
    if all(task.status()['state'] in ('COMPLETED', 'FAILED') for task in tasks):
        break
    time.sleep(10)
print(f"Export işlemleri tamamlandı!")

Export işlemleri tamamlandı!


In [ ]:
!pip install nbstripout > /dev/null 2>&1

In [ ]:
!nbstripout /content/drive/MyDrive/GitHub_Repos/urban-heat-island/notebooks/01_data_acquisition_geemap.ipynb